In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np

import PIL

import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils import data

from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import ProjectedGradientDescentPyTorch
from art.estimators.classification import PyTorchClassifier

import matplotlib.pyplot as plt
import time
import logging
import datetime
import random

cuda = torch.cuda.is_available()
cuda

True

In [3]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
      res = tensor + torch.randn(tensor.size()) * self.std + self.mean
      return torch.clamp(input=res, min=-0.5, max=0.5)
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [4]:
class VGG16(nn.Module):
    def __init__(self, nb_classes=10):
        super(VGG16, self).__init__()
        self.vgg16 = models.vgg16_bn()
        self.linear = nn.Linear(1000, nb_classes)
        
    def forward(self, x):
        x = self.vgg16(x)
        x = self.linear(x)
        return x

In [5]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device
        
        with torch.cuda.amp.autocast():
          outputs = model(data)
          loss = criterion(outputs, target)
          running_loss += loss.item()

          _, predicted = torch.max(outputs.data, 1)
          total_predictions += target.size(0)
          correct_predictions += (predicted == target).sum().item()
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # loss.backward()
        # optimizer.step()

        del data
        del target
    
    
    running_loss /= len(train_loader)
    acc = (correct_predictions/total_predictions)*100.0
    logging.info('Training Loss: {}'.format(running_loss))
    logging.info('Training Accuracy: {}%'.format(acc))
    return running_loss


def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(test_loader):   
            data = data.to(device)
            target = target.to(device)

            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()

            loss = criterion(outputs, target).detach()
            running_loss += loss.item()

            # with torch.cuda.amp.autocast():
            #   outputs = model(data)

            #   _, predicted = torch.max(outputs.data, 1)
            #   total_predictions += target.size(0)
            #   correct_predictions += (predicted == target).sum().item()

            #   loss = criterion(outputs, target).detach()
            #   running_loss += loss.item()
            

            del data
            del target

        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        logging.info('Testing Loss: {}'.format(running_loss))
        logging.info('Testing Accuracy: {}%'.format(acc))
        return running_loss, acc

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

In [6]:
# configure logging
logger = logging.getLogger("")

# reset handler
for handler in logging.root.handlers[:]:
  logging.root.removeHandler(handler)

# set handler
stream_hdlr = logging.StreamHandler()
file_hdlr = logging.FileHandler('./gdrive/My Drive/IDL_Project/log_{}.log'.format(datetime.datetime.now()))

formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
stream_hdlr.setFormatter(formatter)
file_hdlr.setFormatter(formatter)

logger.addHandler(stream_hdlr)
logger.addHandler(file_hdlr)

logger.setLevel(logging.INFO)

In [7]:
train_batchsize = 128
test_batchsize = 100
num_workers = 8
num_classes = 10

n_epochs = 30
img_size = 224
lr = 1e-4
min_lr = 1e-8
weight_decay = 5e-4
num_models = 10
noise_std = 0.06

device = torch.device("cuda" if cuda else "cpu")

hyper_params = {'lr': lr, 'min_lr': min_lr, 'weight_decay': weight_decay, 'num_models': num_models, 'num_epochs': n_epochs, 'noise_std': noise_std}
logging.info(hyper_params)


2020-11-04 02:04:37,009 INFO {'lr': 0.0001, 'min_lr': 1e-08, 'weight_decay': 0.0005, 'num_models': 10, 'num_epochs': 30, 'noise_std': 0.06}


# Training

In [ ]:
seeds = np.arange(0, num_models)

for seed in seeds:
  logging.info('Seed: {}'.format(seed))
  torch.manual_seed(seed)

  train_transform = transforms.Compose([transforms.Resize(size=img_size),
                      transforms.ColorJitter(hue=.05, saturation=.05),
                      transforms.RandomHorizontalFlip(p=0.5),
                      transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
                      transforms.ToTensor(),
                      transforms.Normalize((0, 0, 0), (1, 1, 1)),
                      AddGaussianNoise(0., noise_std)])

  test_transform = transforms.Compose([transforms.Resize(size=img_size),
                     transforms.ToTensor(),
                     transforms.Normalize((0, 0, 0), (1, 1, 1)),
                     AddGaussianNoise(0., noise_std)])

  
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_batchsize, shuffle=True, num_workers=num_workers)
  testloader = torch.utils.data.DataLoader(testset, batch_size=test_batchsize, shuffle=False, num_workers=num_workers)

  # model = models.vgg16()
  model = VGG16(num_classes)
  model.apply(init_weights)
  criterion = nn.CrossEntropyLoss()
  # optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=False)
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, cooldown=5, min_lr=min_lr, verbose=True)
  scaler = torch.cuda.amp.GradScaler()
  model.to(device)

  # Train_loss = []
  # Test_loss = []
  # Test_acc = []

  for i in range(n_epochs):
    tic = time.time()
    logging.info('Epoch: {}'.format(i))
    train_loss = train_epoch(model, trainloader, criterion, optimizer)
    test_loss, test_acc = test_model(model, testloader, criterion)
    scheduler.step(test_loss)
    # Train_loss.append(train_loss)
    # Test_loss.append(test_loss)
    # Test_acc.append(test_acc)
    toc = time.time()
    logging.info('Time: {}s'.format(toc - tic))
    logging.info('='*20)
    
  torch.save({'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()}, 
        "gdrive/My Drive/IDL_Project/Model_{}".format(seed))
  
  torch.cuda.empty_cache()
  del model
  del criterion
  del optimizer
  del scheduler
  del scaler
  del trainloader
  del testloader
  del train_transform
  del test_transform

  


# Ensemble Class Definition

In [69]:
class MyEnsemble(nn.Module):
    def __init__(self, model_list, num_models, num_models_selected, num_classes=10):
        super(MyEnsemble, self).__init__()
        self.model_list = []
        self.num_models = num_models
        self.num_models_selected = num_models_selected
        self.num_classes = num_classes
        for model in model_list:
          self.model_list.append(model)
        # Remove last linear layer
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        # mask = torch.randperm(num_models)[:num_models_selected]
        indices = [i for i in range(self.num_models)]
        random.shuffle(indices)
        indices = indices[:self.num_models_selected]
        models_selected = [self.model_list[idx] for idx in indices]

        logits_list = []
        labels_list = []

        noise = torch.randn(x.size()) * noise_std
        noise = noise.to(device)
        x_noised = x + noise

        for model in models_selected:
          logits = model(x_noised)
          label_out = torch.unsqueeze(torch.argmax(self.softmax(logits), dim=1), dim=0)
          logits = torch.unsqueeze(logits, dim=0)
          labels_list.append(label_out)
          logits_list.append(logits)
        
        # majority vote
        labels_tensor = torch.cat(labels_list, dim=0)
        logits_tensor = torch.cat(logits_list, dim=0)
        voted_class = torch.max(labels_tensor, dim=0) # possibly ties
        mask = (labels_tensor == voted_class.values).int()

        mask = mask.unsqueeze(dim=1).repeat(1, num_classes, 1)
        mask = mask.transpose(1, 2)

        sum_logits = torch.sum(mask * logits_tensor, dim=0)
        divider = torch.sum(mask, dim=0)

        return sum_logits / divider

        # max_logits = torch.max(mask * logits_tensor, dim=0)
        # return max_logits.values


# Ensemble Parameters

In [48]:
num_models = 9
num_models_selected = 2
num_classes = 10

# Load Saved Models

In [10]:
model_list = []
for i in range(4, num_models):
  model = VGG16()
  model_data = torch.load('gdrive/My Drive/IDL_Project/Model_{}'.format(i))
  model.load_state_dict(model_data['model_state_dict'])
  model.to(device)
  model_list.append(model)

# Create Ensemble Object

In [70]:
my_ensemble = MyEnsemble(model_list, num_models, num_models_selected, num_classes=10)
my_ensemble.to(device)

MyEnsemble(
  (softmax): Softmax(dim=None)
)

# Install ART

In [ ]:
!pip install adversarial-robustness-toolbox

# Create classifier object

In [88]:
classifier = PyTorchClassifier(
    model=my_ensemble,
    clip_values=(-0.5, 0.5),
    loss=criterion,
    input_shape=(3, 224, 224),
    nb_classes=10,
)

2020-11-04 03:01:01,201 INFO Inferred 1 hidden layers on PyTorch classifier.


# Create attack object

In [89]:
# attack = FastGradientMethod(estimator=classifier, eps=0.01)
attack = ProjectedGradientDescentPyTorch(estimator=classifier, eps=0.1, eps_step=0.01)

# Get test examples

In [100]:
x_test = []
y_test = []
for batch_idx, (X, Y) in enumerate(testloader):
  x_test.append(X.squeeze().numpy())
  y_test.append(Y[0].item())
x_test = np.array(x_test)
print(x_test.shape)

(10000, 3, 224, 224)


In [95]:
test_transform = transforms.Compose([transforms.Resize(size=img_size),
                     transforms.ToTensor(),
                     transforms.Normalize((0, 0, 0), (1, 1, 1))])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=num_workers)


Files already downloaded and verified


# Generating attack samples

In [90]:
x_test_adv = attack.generate(x=x_test[:100])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
PGD - Random Initializations: 100%|██████████| 1/1 [03:27<00:00, 207.44s/it]
2020-11-04 03:04:45,260 INFO Success rate of attack: 93.00%


In [21]:
for i in range(len(y_test)):
  y_test[i] = y_test[i].item()

In [91]:
predictions = classifier.predict(x_test_adv)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [92]:
accuracy = np.sum(np.argmax(predictions, axis=1) == np.array(y_test)[:100]) / 100

In [93]:
accuracy

0.07

In [ ]:
model0 = VGG16()
model_data = torch.load('gdrive/My Drive/IDL_Project/Model_0')
model0.load_state_dict(model_data['model_state_dict'])
model0.to(device)

In [78]:
test_model(model=model0, test_loader=testloader, criterion=criterion)

2020-11-04 02:51:57,260 INFO Testing Loss: 0.3685084188319378
2020-11-04 02:51:57,261 INFO Testing Accuracy: 88.17%


(0.3685084188319378, 88.17)